In [5]:
import os
import time
import requests
import pandas as pd
#from tqdm import trange  # Progress bar for Jupyter notebooks

# ✅ Set up Mistral AI API Key
MISTRAL_API_KEY = "***REMOVED***"
API_URL = "https://api.mistral.ai/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {MISTRAL_API_KEY}", "Content-Type": "application/json"}

# ✅ Define video file paths (Since Mistral AI does NOT support video input, we assume text-based metadata)
VIDEO_PATH = 'tesla-real-world-video-q-a/videos/videos/'
video_names = [video for video in os.listdir(VIDEO_PATH) if video.endswith('.mp4')]



In [6]:
# ✅ Ensure 'answers' column exists
csv_file = "tesla-real-world-video-q-a/questions.csv"
questions = pd.read_csv(csv_file)
if "answers" not in questions.columns:
    questions["answers"] = None

# ✅ Process Each Question
total_questions = len(questions)

for index, row in questions.iterrows():
    if pd.notna(row["answers"]):  # Skip if already answered
        continue

    question_text = row["question"]

    # ✅ Define Prompt for Mistral AI
    prompt = f"""
    You are an expert in traffic and driving laws.
    Carefully analyze the following driving scenario and answer the question with reasoning.

    Question: {question_text}
    Answer:
    """

    answered = False  # Retry flag

    while not answered:
        try:
            # ✅ Call Mistral AI API
            payload = {
                "model": "mistral-tiny",
                "messages": [
                    {"role": "system", "content": "You are an expert in analyzing traffic situations."},
                    {"role": "user", "content": prompt}
                ]
            }
            response = requests.post(API_URL, json=payload, headers=HEADERS)
            response.raise_for_status()  # Raise error if request fails
            
            # ✅ Extract Response
            result = response.json()["choices"][0]["message"]["content"]
            questions.at[index, "answers"] = result  # Update DataFrame
            
            answered = True  # Mark success
            
        except requests.exceptions.RequestException as e:
            print(f"Error on question {row['id']}: {e}")
            time.sleep(10)  # Retry delay

    # ✅ Print Progress
    print(f"Processed {index+1}/{total_questions} questions...")

# ✅ Save updated CSV with answers
output_file = "updated_questions.csv"
questions.to_csv(output_file, index=False)
print(f"✅ Processing complete! Answers saved to '{output_file}'.")

Processed 1/50 questions...
Processed 2/50 questions...
Processed 3/50 questions...
Processed 4/50 questions...
Processed 5/50 questions...
Processed 6/50 questions...
Processed 7/50 questions...
Processed 8/50 questions...
Processed 9/50 questions...
Processed 10/50 questions...
Processed 11/50 questions...
Processed 12/50 questions...
Processed 13/50 questions...
Processed 14/50 questions...
Processed 15/50 questions...
Processed 16/50 questions...
Processed 17/50 questions...
Processed 18/50 questions...
Processed 19/50 questions...
Processed 20/50 questions...
Processed 21/50 questions...
Processed 22/50 questions...
Processed 23/50 questions...
Processed 24/50 questions...
Processed 25/50 questions...
Processed 26/50 questions...
Processed 27/50 questions...
Processed 28/50 questions...
Processed 29/50 questions...
Processed 30/50 questions...
Processed 31/50 questions...
Processed 32/50 questions...
Processed 33/50 questions...
Processed 34/50 questions...
Processed 35/50 questio

In [7]:
# ✅ Loop through first 50 questions
for question_number in trange(50):  
    # Skip if answer is already filled
    if questions.iloc[question_number]['answers'] not in [None, "nan", ""]:
        continue
    
    # Get video filename (Assuming video metadata is used as context)
    video_metadata = f"Video filename: {video_names[question_number]}"

    # Get question
    question = questions.iloc[question_number]['question']
    
    # ✅ Define prompt for Mistral AI
    prompt = f"""
    First, summarize the key events in the following video: {video_metadata}.
    Then, reason through the following question carefully.
    Lastly, provide the correct answer and specify which character is correct. 
    (Ego refers to the current driver.)

    Question: {question}
    Answer:
    """

    answered = False  # Flag to retry on failure
    
    while not answered:
        try:
            # ✅ Call Mistral AI API
            payload = {
                "model": "mistral-tiny",
                "messages": [{"role": "system", "content": "You are an AI assistant analyzing driving videos."},
                             {"role": "user", "content": prompt}]
            }
            response = requests.post(API_URL, json=payload, headers=HEADERS)
            response.raise_for_status()  # Raise error if request fails
            
            # ✅ Extract response text
            result = response.json()["choices"][0]["message"]["content"]
            questions.at[question_number, 'answers'] = result  # Correct way to update Pandas DataFrame
            
            answered = True  # Mark as successfully answered
            
        except requests.exceptions.RequestException as e:
            print(f"Error on question {question_number}: {e}")
            time.sleep(10)  # Wait before retrying

# ✅ Save updated CSV with answers
questions.to_csv("updated_questions.csv", index=False)
print("✅ Processing complete! Answers saved to 'updated_questions.csv'.")


NameError: name 'trange' is not defined

In [45]:
import pandas as pd

questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions.head()

,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,None
1,2,Where can ego legally park on this street? A. ...,None
2,3,What is the best description of the maneuver e...,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None
4,5,"What is the blinker state of the oncoming car,...",None


In [ ]:
from IPython.display import Markdown
import time
from tqdm import trange


for question_number in trange(50):
    if questions.iloc[question_number]['answers'] is not None:
        continue
    video_name = video_names[question_number][1:]
    video_file = client.files.get(name=video_name)
    question = questions.iloc[question_number]['question']
    
    prompt = f"First, explain what's going on in the video. Then, reason through the following question. Lastly, provide the correct answer to the question, make sure to specify which character is correct. Ego refers to the current driver. Question: {question}\nAnswer:"
    
    answered = False
    
    while not answered:
        try: 
            result = client.models.generate_content(
                model="gemini-1.5-pro",
                contents=[
                    video_file,
                    prompt
            ]).text

            questions.iloc[question_number]['answers'] = result
            answered = True
            
        except Exception as e:
            print(f"Error on question {question_number}: {e}")
        
        time.sleep(10)

  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/d8/6961hjxn69l6pchnksn1_83w0000gn/T/ipykernel_22273/1037324794.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions.iloc[question_number]['answers'] = result
 28%|██▊       | 14/50 [02:15<09:21, 15.60s/it]

Error on question 14: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 40%|████      | 20/50 [04:10<08:49, 17.65s/it]

Error on question 20: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 46%|████▌     | 23/50 [05:09<08:12, 18.23s/it]

Error on question 23: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


 48%|████▊     | 24/50 [05:38<09:15, 21.37s/it]

Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on question 24: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error on q

In [54]:
questions

,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,A
1,2,Where can ego legally park on this street? A. ...,B
2,3,What is the best description of the maneuver e...,C
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,A
4,5,"What is the blinker state of the oncoming car,...",A
5,6,What is the reason ego changed lanes to the le...,B
6,7,What is the correct behavior for ego and why? ...,None
7,8,How many flashing traffic light bulbs are ther...,None
8,9,Which one if more accurate in terms of the dis...,None
9,10,"Ignoring pedestrian lights, how many traffic l...",None
